In [3]:
import os

print("原始工作路径:", os.getcwd())
os.chdir("/workspace/")
print("新工作路径:", os.getcwd())

原始工作路径: /workspace
新工作路径: /workspace


In [4]:
!apt-get -y update
!apt-get -y upgrade
!apt-get install -y cmake
!apt-get install -y libcurl4-gnutls-dev

Get:1 http://mirrors.cloud.tencent.com/ubuntu focal InRelease [265 kB]
Hit:2 http://mirrors.cloud.tencent.com/ubuntu focal-security InRelease         
Hit:3 http://mirrors.cloud.tencent.com/ubuntu focal-updates InRelease          
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Fetched 265 kB in 1s (463 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Calculating upgrade... Done
The following packages have been kept back:
  libcudnn9-cuda-12 libcudnn9-dev-cuda-12 libnccl-dev libnccl2
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Reading package lists... Done


In [5]:
%pip install --upgrade pip
%pip install --upgrade ipywidgets

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#用 Unsloth 库的优化方案，以4位量化+梯度检查点的方式，低显存加载高性能的 Qwen2.5-14B 指令微调模型
from unsloth import FastLanguageModel
import torch

# 调用Unsloth封装的from_pretrained方法，加载预训练模型和配套分词器
# 返回值：model为量化后的Qwen2.5-14B模型实例，tokenizer为模型配套的分词器实例
model, tokenizer = FastLanguageModel.from_pretrained(
    # 指定加载的模型名称/路径：Unsloth优化的Qwen2.5-14B指令微调版（bnb-4bit表示4位量化版本）
    # 该模型专为对话/指令执行场景设计，开箱即用4位量化，降低显存门槛
    model_name = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit",
    # 启用4位量化加载模型（基于bitsandbytes实现），显存占用比FP16降低75%（14B模型仅需8-10GB显存）
    load_in_4bit = True,
    # 启用梯度检查点（Gradient Checkpointing）：反向传播时不保存所有中间激活值，以计算换显存
    # 进一步降低训练/推理时的显存消耗，对14B级大模型至关重要
    use_gradient_checkpointing = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 为已加载的量化模型配置LoRA（低秩适配）微调策略，返回适配后的模型实例
# FastLanguageModel.get_peft_model是Unsloth封装的PEFT优化方法，专为大模型低显存微调设计
model = FastLanguageModel.get_peft_model(
   model,  # 传入之前加载的Qwen2.5-14B量化模型实例
   r = 16,  # LoRA低秩矩阵的秩（核心超参数），控制低秩矩阵的维度；r越大可拟合的信息越多，但显存占用略增
            # 16是Qwen2.5-14B的推荐值，平衡效果与显存
   # 指定对模型哪些模块注入LoRA参数（Qwen2.5基于Transformer架构，核心注意力/前馈层）
   # q/k/v/o_proj：注意力层的查询/键/值/输出投影层；gate/up/down_proj：前馈网络的核心层
   # 仅对这些核心层微调，既保证效果又减少参数量
   target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
   lora_alpha = 16,  # LoRA的缩放系数，与r配合使用（建议和r取值相同），用于平衡LoRA参数的更新幅度
   lora_dropout = 0,  # LoRA层的Dropout概率，0表示不使用Dropout（小数据集/指令微调场景推荐0，避免过拟合）
   bias = "none",     # 是否训练模型的偏置参数："none"表示不训练任何偏置，进一步减少训练参数量（主流选择）
   # 启用Unsloth定制的梯度检查点优化（"unsloth"模式），相比原生PyTorch版本显存占用再降20%-30%
   # 原理：更精细地控制激活值的保存/重计算，适配LoRA微调的显存特性
   use_gradient_checkpointing = "unsloth", 
   random_state = 3407,  # 随机种子，固定种子保证实验可复现（3407是LLM微调的经典默认值）
   use_rslora = False,   # 是否启用RSLoRA（秩稳定LoRA）：False表示使用标准LoRA（Qwen2.5-14B无需RSLoRA）
   loftq_config = None,  # 是否启用LoftQ（量化感知LoRA初始化）：None表示不启用（4bit量化已优化，无需额外初始化）
)

Unsloth 2025.11.3 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [8]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='med-train-dateset.json', split='train')
chat_template = """下面是一些描述一些任务的说明。编写适当完成每个请求的响应。

### 指令:
{INPUT}

### 回应:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
   
)
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/119397 [00:00<?, ? examples/s]

{'conversations': [{'content': '怀孕37周我憋的上宝宝胎动多胎心异常怎么回事', 'role': 'user'}, {'content': '胎动频繁，胎心快，要考虑胎儿是不是缺氧的表现。建议去医院做胎心监护，如果评分低于9分，要吸氧等处理。', 'role': 'assistant'}], 'text': '下面是一些描述一些任务的说明。编写适当完成每个请求的响应。\n\n### 指令:\n怀孕37周我憋的上宝宝胎动多胎心异常怎么回事\n\n### 回应:\n胎动频繁，胎心快，要考虑胎儿是不是缺氧的表现。建议去医院做胎心监护，如果评分低于9分，要吸氧等处理。<|im_end|>'}


In [ ]:
# 导入TRL库的SFTTrainer类：专为大语言模型（LLM）监督微调设计的训练器
# 封装了SFT的核心逻辑（如序列长度处理、数据集打包、LoRA适配），比原生Trainer更适配LLM微调
from trl import SFTTrainer
# 导入Transformers的TrainingArguments类：用于定义所有训练相关的超参数（批次、学习率、步数等）
from transformers import TrainingArguments
# 导入Unsloth的硬件检测工具函数：检测当前GPU是否支持bfloat16精度，用于自动选择混合精度策略
from unsloth import is_bfloat16_supported

# 初始化SFT训练器，整合模型、分词器、数据集和训练参数
trainer = SFTTrainer(
    model = model,  # 传入已配置LoRA的Qwen2.5-14B量化模型实例
    tokenizer = tokenizer,  # 传入模型配套的分词器实例，用于文本token化
    train_dataset = dataset,  # 传入预处理后的训练数据集（需包含"text"字段，存储待微调的指令/文本）
    dataset_text_field = "text",  # 指定数据集中存储训练文本的字段名（需与数据集结构匹配）
    max_seq_length = 2048,  # 设定模型输入的最大序列长度（Qwen2.5-14B支持4096，2048平衡显存与效果）
    dataset_num_proc = 2,  # 数据集预处理的并行进程数（2适配消费级CPU，加速数据集加载/分词）
    packing = False,  # 是否启用数据集打包：False表示不打包（指令微调场景推荐False，避免上下文混乱）
                      # 打包会将短文本拼接成固定长度，适合大文本语料，指令微调无需此操作
    # 定义核心训练超参数，通过TrainingArguments实例传入
    args = TrainingArguments(
        per_device_train_batch_size = 2,  # 单GPU的训练批次大小（2适配10GB+显存，可根据显存调整）
        gradient_accumulation_steps = 4,  # 梯度累积步数：将4个批次的梯度累积后再更新参数
                                          # 等效于总批次大小=2*4=8，既保证批次大小又降低单步显存占用
        warmup_steps = 5,  # 学习率预热步数：前5步学习率从0线性升至设定值，避免初始学习率过大导致训练不稳定
        max_steps = 60,  # 最大训练步数（替代epochs，更易控制微调时长）：60步适合小数据集（如千级样本）
        learning_rate = 2e-4,  # LoRA微调的学习率（2e-4是Qwen2.5-14B+LoRA的经典推荐值）
        fp16 = not is_bfloat16_supported(),  # 是否启用FP16混合精度：硬件不支持BF16时自动启用FP16
        bf16 = is_bfloat16_supported(),  # 是否启用BF16混合精度：A100/RTX 3090/4090等新GPU支持，精度更高、速度更快
        logging_steps = 1,  # 日志打印步数：每训练1步打印一次损失、学习率等信息，便于监控训练状态
        optim = "adamw_8bit",  # 启用8位量化的AdamW优化器：相比原生AdamW显存占用降低50%，不损失训练效果
        weight_decay = 0.01,  # 权重衰减系数：0.01用于抑制过拟合，平衡模型泛化能力
        lr_scheduler_type = "linear",  # 学习率调度器：线性衰减（预热后学习率线性降至0），适配LoRA微调
        seed = 3407,  # 随机种子：固定种子保证训练过程可复现（LLM微调通用值）
        output_dir = "outputs",  # 训练结果保存路径：存储模型权重、日志、检查点等
        report_to = "none",  # 禁用第三方日志工具（如WandB）：仅本地打印日志，简化训练流程
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/119397 [00:00<?, ? examples/s]

[accelerate.utils.other|WARNING]Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 119,397 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.355300
2,3.859300
3,3.700000
4,3.089900
5,3.037700
6,2.975000
7,3.011800
8,2.610900
9,2.357000
10,2.298500


In [ ]:
# 将模型切换为推理模式（Unsloth封装的优化方法）
# 自动禁用训练相关的梯度计算、dropout等，启用推理优化（如KV缓存），提升生成速度、降低显存占用
FastLanguageModel.for_inference(model) 

# 定义对话历史/用户输入，遵循Qwen2.5的对话格式（role: user/assistant）
# 列表中每个字典代表一轮对话，此处仅包含用户的提问：“我肚子疼”
messages = [                   
    {"role": "user", "content": "我肚子疼"},
]

# 使用分词器的对话模板处理输入，转换为模型可识别的token ID（张量格式）
input_ids = tokenizer.apply_chat_template(
    messages,  # 传入对话列表
    add_generation_prompt = True,  # 自动添加“助手回复前缀”（Qwen2.5要求的格式），告诉模型接下来要生成助手回复
    return_tensors = "pt",  # 返回PyTorch张量（pt），适配模型的张量计算
).to("cuda")  # 将输入张量移至GPU（cuda），与模型的运行设备一致

# 导入Transformers的TextStreamer类：实现流式输出（逐字/逐句返回回复，而非一次性输出）
from transformers import TextStreamer
# 初始化流式输出器，绑定分词器；skip_prompt=True表示跳过输入提示词（仅输出模型生成的回复）
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

# 调用模型的generate方法生成回复，流式输出结果
# _ = 忽略返回值（因streamer已实时打印回复，无需额外存储生成的token ID）
_ = model.generate(
    input_ids,  # 传入处理后的输入张量
    streamer = text_streamer,  # 绑定流式输出器，实时打印回复
    max_new_tokens = 128,  # 限制生成的最大新token数（128足够覆盖日常对话回复长度）
    pad_token_id = tokenizer.eos_token_id,  # 指定padding token为EOS（结束符），避免生成过程中出现padding错误
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


可能是肠痉挛引起的，建议服用山莨菪碱片治疗<|im_end|>


In [12]:
model.save_pretrained_merged("qwen25-14b-offical-finetuned-bnb-4bit", tokenizer, save_method="merged_4bit_forced",)

Unsloth: Merging LoRA weights into 4bit model...


/root/miniforge3/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Unsloth: Merging finished.
Unsloth: Found skipped modules: ['lm_head']. Updating config.
Unsloth: Saving merged 4bit model to qwen25-14b-offical-finetuned-bnb-4bit...
Unsloth: Merged 4bit model saved.
Unsloth: Merged 4bit model process completed.
